<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/PartisanBiasDetection/NewsRSS_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Dependancies
!pip install requests beautifulsoup4 --quiet

#Import libraries
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# @title Define Utility Functions

#Article Scrapper
def scrape_article_text(url):
    # Send a GET request to the URL
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find elements containing text - this part may need customization
        # For example, assuming article text is within <p> tags
        article_text = ' '.join(p.get_text() for p in soup.find_all('p'))
        return article_text
    else:
        return "Error: Unable to fetch the webpage."

#Query for LLM
def query(payload, headers):
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()  # Expecting this to always be a list for simplicity
    else:
        return [{"error": response.text}]

#Political Bias Classification using LLM
def reduce_and_query(initial_text, llm_instructions, reduction_fraction, headers):
    text = initial_text
    while True:
        payload = {"inputs": llm_instructions + text}
        response = query(payload, headers)
        # Since we expect a list, check the first item for an 'error' key
        if "error" in response[0]:
            error_message = response[0]["error"]
            if "token" in error_message.lower():
                # Reduce the text by the specified fraction if it's a token limit error
                new_length = int(len(text) * (1 - reduction_fraction))
                text = text[:new_length]
            else:
                # Return the error message directly if it's a different kind of error
                return error_message
        else:
            # If there's no 'error' key, assume the response is successful
            return response

In [21]:
# CNN Article Link Scraper
page = requests.get('https://www.cnn.com/politics')
soup = BeautifulSoup(page.text, 'html.parser')
allLinks = []
link = "https://www.cnn.com/politics"
for link in soup.find_all('a'):
    allLinks.append(link.get('href'))
allLinks = filter(None, allLinks)
linkEnds = [i for i in allLinks if i.startswith('/2')]
result = ['https://www.cnn.com' + i for i in linkEnds]
CNN_links = set(result)
CNN_links = list(CNN_links)

In [45]:
#Fox News Article Link Scraper
url = 'https://moxie.foxnews.com/google-publisher/politics.xml'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  links.append(y)
FoxNews_links = links

<ipython-input-45-5785ac706a5f>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_page.text, "lxml")


In [63]:
#NYT Article Link Scraper
url = 'https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  links.append(y)
NYT_links = links

<ipython-input-63-d5bd07c2d47a>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_page.text, "lxml")


In [67]:
#Extract article text from links for all news sources
#Use this to text article link efficacy?

url = NYT_links[0]
article_text = scrape_article_text(url)
print(article_text)

 Advertisement By Karen Yourish and  Lazaro Gamio  In an unprecedented moment in modern American history, the 2024 Republican and Democratic presidential candidates will face off in their first debate after just seven weeks of campaigning against each other. The New York Times analyzed what the two candidates have said about each other on social media from July 21, when President Biden dropped out of the race and Vice President Kamala Harris became the frontrunner to replace him as the Democratic nominee, through Sept. 6. (For the most part, their statements on social media mirror their public comments at rallies and other events.) While both candidates attack each other, The Times found that former president Donald Trump targets Ms. Harris much more frequently, an average of more than three times per day, and his posts (on Truth Social) almost always include a personal smear. Ms. Harris’s posts about Mr. Trump (on X) tend not to go for the jugular. A handful of times, she has drawn at

In [ ]:
#List of RSS feeds

#Fox News: Politics
foxPolRSS = f"https://moxie.foxnews.com/google-publisher/politics.xml"
#CNN News: Politics
CNNPolRSS = f"http://rss.cnn.com/rss/cnn_allpolitics.rss"
#NYT News: Politics
NYTPolRSS = f"https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml"
#https://github.com/susannapaoli/web-scraper-nyt

In [ ]:
api_token_hug =


# Following URL is the URL of the LLM being utilized from HuggingFace
API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-xxl"
headers = {"Authorization": f"Bearer {api_token_hug}"}

# Instructions sent to the LLM in front of article text
system_input = "Analyze the text content and assign a label from {left, right, center, uncertain}. In this context, “left” indicates a left-leaning article, “right” signifies a right-leaning article, “center” implies no obvious political leaning, and “uncertain” denotes that the political orientation could not be determined. Please provide your analysis and output a new single line containing only the assigned label."

#reduce by 10% until it fits query
reduction_fraction = 0.10



result = reduce_and_query(article_text, system_input, reduction_fraction, headers)
print(result)

second_article = "https://www.cnn.com/2024/03/19/politics/texas-immigration-law-blocked-appeals/index.html"
second_result = reduce_and_query(second_article, system_input, reduction_fraction, headers)
print(second_result)

third_article = "https://www.cnn.com/videos/world/2024/03/20/israel-gaza-west-bank-settler-movement-clarissa-ward-pkg-intl-ldn-vpx.cnn"
third_result = reduce_and_query(third_article, system_input, reduction_fraction, headers)
print(third_result)

fourth_article = "https://www.cnn.com/2024/03/19/politics/trump-bond-deadline-panic/index.html"
fourth_result = reduce_and_query(fourth_article, system_input, reduction_fraction, headers)
print(fourth_result)

fifth_article = "https://www.foxnews.com/live-news/joe-biden-gop-impeachment-inquiry-hearing-hunter-biden-business-dealings"
fifth_result = reduce_and_query(fifth_article, system_input, reduction_fraction, headers)
print(fifth_result)

sixth_article = "https://www.msnbc.com/deadline-white-house/deadline-legal-blog/trump-supreme-court-immunity-appeal-delay-rcna144155"
sixth_result = reduce_and_query(sixth_article, system_input, reduction_fraction, headers)
print(sixth_result)

seventh_article = "https://www.msnbc.com/rachel-maddow-show/maddowblog/biden-white-house-reason-celebrate-falling-crime-rates-rcna144215"
seventh_result = reduce_and_query(seventh_article, system_input, reduction_fraction, headers)

[{'generated_text': 'left'}]
[{'generated_text': 'uncertain'}]
[{'generated_text': 'center'}]
[{'generated_text': 'center'}]
[{'generated_text': 'center'}]
[{'generated_text': 'center'}]
